In [25]:
import os
import shutil
import pandas as pd
import glob
from tqdm import tqdm
import time
import json
import hmac, hashlib
import requests

In [32]:
import re

# Your string
s = './dataset/1.signal18.csv'

# Regular expression pattern
pattern = r'(?<=/)(\d+)\.'

# Searching for the pattern in the string
match = re.search(pattern, s)

if match:
    number = match.group(1)
    print("Extracted number:", number)
else:
    print("No match found")

Extracted number: 1


In [4]:
ls = glob.glob("./dataset/**/*", recursive=True)
ls

['./dataset/0.signal1.csv', './dataset/1.signal1.csv']

In [6]:
for i in tqdm(range(2,100)):
    shutil.copyfile('./dataset/0.signal1.csv', f'./dataset/0.signal{i}.csv')
    shutil.copyfile('./dataset/1.signal1.csv', f'./dataset/1.signal{i}.csv')

100%|██████████| 98/98 [00:00<00:00, 1808.57it/s]


## Uploading to EI

In [29]:
def upload_ei(_name_label, _values, hmac_key="c8fbafed72b829d4eec44d5f9afd486b", api_key="ei_d54e2d9fbff13d1dc02b6fe7850504b688bfd083c6e4a79c8ba639f3ba80171b"):
    
    HMAC_KEY = hmac_key #"479df84bf18b1722a5f6fcb094265f98"
    API_KEY = api_key #"ei_3d1d5dce11f879e02009083dc58f828de7f450e550dee0d503acca07e110e4db"
    
    emptySignature = ''.join(['0'] * 64)

    Fs=1000
    Ts=(1/Fs)*1000

    data = {
        "protected": {
            "ver": "v1",
            "alg": "HS256",
            "iat": time.time() # epoch time, seconds since 1970
        },
        "signature": emptySignature,
        "payload": {
            "device_name": "ac:87:a3:0a:2d:1b",
            "device_type": "NANO33BLE",
            "interval_ms": Ts,
            "sensors": [
                { "name": "Volts", "units": "adu/mv" },
                #{ "name": "Volts", "units": "adu/mv" },
                #{ "name": "Volts", "units": "adu/mv" }
            ],
            "values": _values.tolist()
        }
    }

    # encode in JSON
    encoded = json.dumps(data)

    # sign message
    signature = hmac.new(bytes(HMAC_KEY, 'utf-8'), msg = encoded.encode('utf-8'), digestmod = hashlib.sha256).hexdigest()

    # set the signature again in the message, and encode again
    data['signature'] = signature
    encoded = json.dumps(data)

    # and upload the file
    res = requests.post(url='https://ingestion.edgeimpulse.com/api/training/data',
                        data=encoded,
                        headers={
                            'Content-Type': 'application/json',
                            'x-file-name': _name_label,
                            'x-api-key': API_KEY
                        })
    # if (res.status_code == 200):
    #     print('Uploaded file to Edge Impulse', res.status_code, res.content)
    # else:
    #     print('Failed to upload file to Edge Impulse', res.status_code, res.content)

In [27]:
ls = glob.glob("./dataset/**/*", recursive=True)
ls

['./dataset/0.signal18.csv',
 './dataset/1.signal76.csv',
 './dataset/1.signal29.csv',
 './dataset/1.signal91.csv',
 './dataset/0.signal73.csv',
 './dataset/1.signal10.csv',
 './dataset/1.signal39.csv',
 './dataset/1.signal81.csv',
 './dataset/1.signal8.csv',
 './dataset/0.signal1.csv',
 './dataset/0.signal76.csv',
 './dataset/0.signal82.csv',
 './dataset/0.signal98.csv',
 './dataset/0.signal69.csv',
 './dataset/0.signal23.csv',
 './dataset/0.signal77.csv',
 './dataset/1.signal93.csv',
 './dataset/0.signal47.csv',
 './dataset/1.signal86.csv',
 './dataset/0.signal21.csv',
 './dataset/0.signal29.csv',
 './dataset/1.signal38.csv',
 './dataset/0.signal84.csv',
 './dataset/0.signal27.csv',
 './dataset/0.signal80.csv',
 './dataset/1.signal73.csv',
 './dataset/1.signal77.csv',
 './dataset/1.signal26.csv',
 './dataset/0.signal65.csv',
 './dataset/1.signal57.csv',
 './dataset/0.signal64.csv',
 './dataset/1.signal87.csv',
 './dataset/1.signal3.csv',
 './dataset/1.signal48.csv',
 './dataset/0.sig

In [8]:


df = pd.read_csv("0.signal1.csv", delimiter="\t")
df.head()

,timestamp,inicio
0,0,-0.000138
1,1,-0.000544
2,2,-0.000344
3,3,0.001138
4,4,0.001554


In [19]:
data_raw = df["inicio"].values.reshape(-1,1)#.tolist()

In [26]:
upload_ei("0", data_raw)

Uploaded file to Edge Impulse 200 b'0.4fccv03c.json'


In [33]:
for i in tqdm(ls):
    match = re.search(pattern, i)
    if match:
        number = match.group(1)
        #print("Extracted number:", number)
    upload_ei(str(number), pd.read_csv(i, delimiter="\t")["inicio"].values.reshape(-1,1))

 43%|████▎     | 85/198 [03:48<05:03,  2.69s/it]


KeyboardInterrupt: 